In [26]:
import math

class Value:
    def __init__(self, data, _children=(), _op='', label=''):
        self.data = data
        self.grad = 0.0
        self._backward = lambda: None  # Placeholder until a proper backward function is set
        self._prev = set(_children)    # References to input nodes
        self._op = _op                 # Operation that produced this value
        self.label = label             # Optional label for debugging or visualization

    def __repr__(self):
        return f"Value(data={self.data})"

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')

        def _backward():
            # Gradient of addition is distributed equally
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad

        out._backward = _backward
        return out
    def __sub__(self, other):
        return self + (-other)  # Reuse __add__ by negating other
    def __neg__(self):
       return self * -1  # Negate the value

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')

        def _backward():
            # d(out)/d(self) = other.data
            # d(out)/d(other) = self.data
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad

        out._backward = _backward
        return out
    def __rmul__(self, other): 
        # __rmul__: Called when b * a and b doesn’t know how to multiply a
        return self * other

    # a/b 
    # a * (a/b)
    # a * (b**-1)
    def __pow__(self, other):
        assert isinstance(other, (int, float)), "Exponent must be a number"
        out = Value(self.data ** other, (self,), f'**{other}')
        def _backward():
            # Derivative of x^n is n*x^(n-1)
            self.grad += other * (self.data ** (other - 1)) * out.grad
        out._backward = _backward
        return out

        


    def __truediv__(self, other):
        return self * (other** -1)
    

    def tanh(self):
        x = self.data
        t = (math.exp(2 * x) - 1) / (math.exp(2 * x) + 1)  # tanh(x)
        out = Value(t, (self,), 'tanh')

        def _backward():
            # Derivative of tanh is 1 - tanh(x)^2
            self.grad += (1 - t**2) * out.grad

        out._backward = _backward
        return out


    def exp(self):
        x = self.data
        out = Value(math.exp(x), (self,), 'exp')
        def _backward():
            # Derivative of exp(x) is exp(x)
            self.grad += out.data * out.grad
        out._backward = _backward
        return out



    def backward(self):
        topo = []         # List of nodes in topological order
        visited = set()

        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)

        build_topo(self)
        self.grad = 1.0    # Seed gradient
        for node in reversed(topo):
            node._backward()


In [27]:
import random

class Neuron : 
    def __init__ (self, nin ):
        self.w = {Value(random.uniform(-1, 1)) for _ in range(nin)}
        self.b = Value(random.uniform(-1, 1))


    def __call__(self, x):
        return 0.0
    


x =[2.0, 3.0, 4.0]
n = Neuron(len(x))
n(x)  # Call the neuron with input x, which should return a Value object

0.0

In [28]:
class Neuron : 
    def __init__ (self, nin ):
        self.w = {Value(random.uniform(-1, 1)) for _ in range(nin)}
        self.b = Value(random.uniform(-1, 1))


    def __call__(self, x):
        act = sum([wi * xi for wi, xi in zip(self.w, x)],self.b)  # Weighted sum of inputs plus bias
        out = act.tanh()  # Use tanh activation function
        return out


# Example usage
x = [2.0, 3.0, 4.0]
n = Neuron(len(x))
result = n(x)  # Call the neuron with input x, which should return a Value object
print(result)  # Print the result of the neuron activation

Value(data=-0.9984966600067113)


In [29]:
class Neuron : 
    def __init__ (self, nin ):
        self.w = {Value(random.uniform(-1, 1)) for _ in range(nin)}
        self.b = Value(random.uniform(-1, 1))


    def __call__(self, x):
        act = sum([wi * xi for wi, xi in zip(self.w, x)],self.b)  # Weighted sum of inputs plus bias
        out = act.tanh()  # Use tanh activation function
        return out
    def parameters(self):
        # Return a list of all parameters (weights and bias) of the neuron
        return list(self.w) + [self.b]

class Layer:
    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]

    def __call__(self, x):
        # Call each neuron with the input x and return a list of outputs
        return [neuron(x) for neuron in self.neurons]
    def parameters(self):
        params = []
        for neuron in self.neurons:
            params.extend(neuron.parameters())
        return params

class MLP :
    def __init__(self, nin, nouts):
        sz = [nin] + nouts  # Input size followed by output sizes for each layer
        self.layers = [Layer(sz[i], sz[i + 1]) for i in range(len(nouts))]

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)  # Pass input through each layer
        return x   
    def parameters(self):
        params = []
        for layer in self.layers:
            params.extend(layer.parameters())
        return params


# Example usage
x = [2.0, 3.0, 4.0]
n = MLP(len(x), [5,5, 3,1])  # Create a multi-layer perceptron with 5 neurons in the first layer and 3 in the second layer
result = n(x)  # Call the MLP with input x, which should return a list of Value objects
print(result)  # Print the result of the MLP activation

[Value(data=-0.4900475283970396)]


In [37]:
xs =[
    [2.0, 3.0, 4.0],
    [1.0, 2.0, 3.0],
    [0.5, 1.5, 2.5],
    [4.0, 5.0, 6.0]
]
ys = [1.0, 0.5, 0.25, 0.1]  # Target outputs for each input
ypred = [n(x) for x in xs]# Get predictions from the MLP for each input
ypred

[[Value(data=-0.29833922573396165)],
 [Value(data=-0.26326122706422006)],
 [Value(data=-0.2930031239133526)],
 [Value(data=-0.46337650830228616)]]

In [31]:
ypred[0][0]

Value(data=-0.4900475283970396)

In [38]:
loss = [(ypred[0]-y)**2 for ypred,y in zip(ypred, ys)]
loss=sum(loss,Value(0.0))  # Sum the losses to get a single loss value
loss.backward()  # Backpropagate the gradients through the network

In [33]:
n.layers[0].neurons[0].parameters()  # Get parameters of the first neuron in the first layer

[Value(data=0.6595016470245281),
 Value(data=0.4979092600927775),
 Value(data=0.5178912218555374),
 Value(data=0.6722986503100894)]

In [34]:
n.layers[0].parameters()  # Get all parameters of the first layer

[Value(data=0.6595016470245281),
 Value(data=0.4979092600927775),
 Value(data=0.5178912218555374),
 Value(data=0.6722986503100894),
 Value(data=-0.5858972822637565),
 Value(data=-0.4276670690418529),
 Value(data=0.3043627137053384),
 Value(data=-0.2441026554839456),
 Value(data=0.7653989723098125),
 Value(data=-0.1655900391045113),
 Value(data=0.03500725978522379),
 Value(data=0.7298160526987942),
 Value(data=-0.9159014840053787),
 Value(data=-0.23338522694850017),
 Value(data=0.4069491345722176),
 Value(data=0.8429830964481739),
 Value(data=0.4130290198471829),
 Value(data=-0.6180900792938067),
 Value(data=0.6344932312885871),
 Value(data=0.4719747108546357)]

In [35]:
for p in n.parameters():  # Get all parameters of the MLP
    print(p.data, p.grad)  # Print each parameter's value and gradient
# Update parameters using gradient descent


0.6595016470245281 0.0015461957342308315
0.4979092600927775 0.004038850403021917
0.5178912218555374 0.006531505071813003
0.6722986503100894 0.0024926546687910863
-0.5858972822637565 0.17068671394945809
-0.4276670690418529 0.35996048904966305
0.3043627137053384 0.549234264149868
-0.2441026554839456 0.18927377510020493
0.7653989723098125 -0.14307986094188835
-0.1655900391045113 -0.2836299328310836
0.03500725978522379 -0.42418000472027884
0.7298160526987942 -0.14055007188919524
-0.9159014840053787 -1.1771072914662688
-0.23338522694850017 -1.868370777292362
0.4069491345722176 -2.559634263118455
0.8429830964481739 -0.691263485826093
0.4130290198471829 -0.31449442370075126
-0.6180900792938067 -0.6124407818941047
0.6344932312885871 -0.910387140087458
0.4719747108546357 -0.2979463581933534
0.14264845426142436 0.13307710907276077
-0.003411386848886089 -0.10212721531741215
-0.5633506711708469 0.11993842390352698
-0.22852770783479448 0.01587394765074554
-0.2550061552800249 0.12516951696624945
-0.

In [39]:
learning_rate = 0.01
for p in n.parameters():
    p.data -= learning_rate * p.grad  # Update each parameter by subtracting the gradient scaled by the learning rate


In [40]:
loss

Value(data=2.880497928505577)

There is some bug in the code above 

In [41]:
class Value:
  
  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"
  
  def __add__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')
    
    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward
    
    return out

  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data * other.data, (self, other), '*')
    
    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
    out._backward = _backward
      
    return out
  
  def __pow__(self, other):
    assert isinstance(other, (int, float)), "only supporting int/float powers for now"
    out = Value(self.data**other, (self,), f'**{other}')

    def _backward():
        self.grad += other * (self.data ** (other - 1)) * out.grad
    out._backward = _backward

    return out
  
  def __rmul__(self, other): # other * self
    return self * other

  def __truediv__(self, other): # self / other
    return self * other**-1

  def __neg__(self): # -self
    return self * -1

  def __sub__(self, other): # self - other
    return self + (-other)

  def __radd__(self, other): # other + self
    return self + other

  def tanh(self):
    x = self.data
    t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
    out = Value(t, (self, ), 'tanh')
    
    def _backward():
      self.grad += (1 - t**2) * out.grad
    out._backward = _backward
    
    return out
  
  def exp(self):
    x = self.data
    out = Value(math.exp(x), (self, ), 'exp')
    
    def _backward():
      self.grad += out.data * out.grad # NOTE: in the video I incorrectly used = instead of +=. Fixed here.
    out._backward = _backward
    
    return out
  
  
  def backward(self):
    
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)
    
    self.grad = 1.0
    for node in reversed(topo):
      node._backward()

In [42]:
from graphviz import Digraph

def trace(root):
  # builds a set of all nodes and edges in a graph
  nodes, edges = set(), set()
  def build(v):
    if v not in nodes:
      nodes.add(v)
      for child in v._prev:
        edges.add((child, v))
        build(child)
  build(root)
  return nodes, edges

def draw_dot(root):
  dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'}) # LR = left to right
  
  nodes, edges = trace(root)
  for n in nodes:
    uid = str(id(n))
    # for any value in the graph, create a rectangular ('record') node for it
    dot.node(name = uid, label = "{ %s | data %.4f | grad %.4f }" % (n.label, n.data, n.grad), shape='record')
    if n._op:
      # if this value is a result of some operation, create an op node for it
      dot.node(name = uid + n._op, label = n._op)
      # and connect this node to it
      dot.edge(uid + n._op, uid)

  for n1, n2 in edges:
    # connect n1 to the op node of n2
    dot.edge(str(id(n1)), str(id(n2)) + n2._op)

  return dot

In [43]:
class Neuron:
  
  def __init__(self, nin):
    self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
    self.b = Value(random.uniform(-1,1))
  
  def __call__(self, x):
    # w * x + b
    act = sum((wi*xi for wi, xi in zip(self.w, x)), self.b)
    out = act.tanh()
    return out
  
  def parameters(self):
    return self.w + [self.b]

class Layer:
  
  def __init__(self, nin, nout):
    self.neurons = [Neuron(nin) for _ in range(nout)]
  
  def __call__(self, x):
    outs = [n(x) for n in self.neurons]
    return outs[0] if len(outs) == 1 else outs
  
  def parameters(self):
    return [p for neuron in self.neurons for p in neuron.parameters()]

class MLP:
  
  def __init__(self, nin, nouts):
    sz = [nin] + nouts
    self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]
  
  def __call__(self, x):
    for layer in self.layers:
      x = layer(x)
    return x
  
  def parameters(self):
    return [p for layer in self.layers for p in layer.parameters()]

In [44]:
x = [2.0, 3.0, -1.0]
n = MLP(3, [4, 4, 1])
n(x)

Value(data=0.9648839166093298)

In [45]:
xs = [
  [2.0, 3.0, -1.0],
  [3.0, -1.0, 0.5],
  [0.5, 1.0, 1.0],
  [1.0, 1.0, -1.0],
]
ys = [1.0, -1.0, -1.0, 1.0] # desired targets

In [48]:
for k in range(20):
  
  # forward pass
  ypred = [n(x) for x in xs]
  loss = sum((yout - ygt)**2 for ygt, yout in zip(ys, ypred))
  
  # backward pass
  for p in n.parameters(): # make sure to reset gradients before backward pass as they accumulate we want to update the parameters based on loss 
    p.grad = 0.0
  loss.backward()
  
  # update
  for p in n.parameters():
    p.data += -0.1 * p.grad
  
  print(k, loss.data)
  

0 0.027607549936140556
1 0.025703823818154933
2 0.024041760092516837
3 0.022578195910621594
4 0.021279719331768213
5 0.02012003411952238
6 0.01907814842431202
7 0.018137095952276555
8 0.017283012897388584
9 0.016504459056418286
10 0.01579191035713806
11 0.015137374055259716
12 0.014534093202274983
13 0.013976317050829905
14 0.013459120814700617
15 0.012978262817475454
16 0.012530070276858011
17 0.01211134724187548
18 0.011719299827109632
19 0.011351475068609784


In [47]:
ypred


[Value(data=0.932969802311823),
 Value(data=-0.9105281817387143),
 Value(data=-0.9083844558273361),
 Value(data=0.9055640134884543)]